# Geospatial Python
This class will be dedicated to work with geospatial information and perform geospatial analysis. We will need a couple of libraries for that. If you use `pip`, you may run the following command (on bash)
```bash
pip install folium
```

In [ ]:
import folium

Create a basic map

In [ ]:
m = folium.Map(location=[4.7110, -74.0721], zoom_start=13)
folium.Marker([4.7110, -74.0721], popup="Bogota").add_to(m)
m

Circles

In [ ]:
folium.CircleMarker(
    location=[4.7110, -74.0721],
    radius=30,
    popup="Circle",
    color="blue",
    fill=True,
    fill_color="cyan"
).add_to(m)
m

We can use a Pandas dataframe to add plenty of places to our map.

In [ ]:
import pandas as pd

In [ ]:
data = {
    'City': ['New York', 'London', 'Tokyo'],
    'Latitude': [40.7128, 51.5074, 35.6895],
    'Longitude': [-74.0060, -0.1278, 139.6917]
}

df = pd.DataFrame(data)

m = folium.Map(location=[20, 0], zoom_start=2)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['City']
    ).add_to(m)

m

Of course, we can include more information:

In [ ]:
# Sample data: major cities
data = {
    'City': ['New York', 'London', 'Tokyo', 'Sydney', 'Cairo'],
    'Lat': [40.7128, 51.5074, 35.6895, -33.8688, 30.0444],
    'Lon': [-74.0060, -0.1278, 139.6917, 151.2093, 31.2357],
    'Population (M)': [8.4, 9.0, 37.4, 5.3, 9.5]
}

df = pd.DataFrame(data)

# Create map centered around Europe
m = folium.Map(location=[30, 0], zoom_start=2)

# Add city markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lon']],
        popup=f"{row['City']} - Pop: {row['Population (M)']}M",
        tooltip=row['City']
    ).add_to(m)

m

We can import example data from the `folium` dataset. You can find all the available datasets here https://github.com/python-visualization/folium/tree/main/examples/data

In [ ]:
import json
import requests

In [ ]:
# Download GeoJSON of world countries
url = 'https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/world-countries.json'
geojson_data = requests.get(url).json()

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=2)

folium.GeoJson(
    geojson_data,
    name='World Countries'
).add_to(m)

m

Choropleth mapping

In [ ]:
# Create fake data for 5 countries
data = {
    'Country': ['FRA', 'BRA', 'IND', 'CHN', 'USA'],
    'Score': [65, 80, 70, 90, 85]
}
df_score = pd.DataFrame(data)

In [ ]:
m = folium.Map(location=[0, 0], zoom_start=2)

folium.Choropleth(
    geo_data=geojson_data,
    data=df_score,
    columns=['Country', 'Score'],
    key_on='feature.id',  # matches ISO3 codes
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Sample Score by Country'
).add_to(m)

m

Now, let us go more local, using graphs.

In [ ]:
import osmnx as ox

In [ ]:
place_name = "Bogotá, Colombia" # Name place
graph = ox.graph_from_place(place_name, network_type='all')

We convert the graph to nodes and edges GeoDataFrames

In [ ]:
nodes, edges = ox.graph_to_gdfs(graph)

and use them to plot the street network.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=0.5, color='black')
ax.set_title('Street Network of Bogotá')
plt.show()

Now let us calculate some (driving) distances on Chapinero.

In [ ]:
# Use a small, specific neighborhood to keep memory low
place_name = "Chapinero, Bogotá, Colombia"

# Get the drivable street network
G = ox.graph_from_place(place_name, network_type='drive', simplify=True)

In [ ]:
# Simple visualization of the graph
ox.plot_graph(G, bgcolor='white', node_size=5, edge_color='gray', edge_linewidth=0.5)

We define two places:

In [ ]:
# Point A: Éxito
orig_lat, orig_lon = 4.6392, -74.0631

# Point B: Somewhere in Chapinero
dest_lat, dest_lon = 4.6483, -74.0586

In [ ]:
# Get nearest nodes to origin and destination points
orig_node = ox.nearest_nodes(G, orig_lon, orig_lat)
dest_node = ox.nearest_nodes(G, dest_lon, dest_lat)

In [ ]:
# Get the shortest distance
# for which we use networkx
import networkx as nx

In [ ]:
shortest_path = nx.shortest_path(G, orig_node, dest_node, weight='length')
length = nx.shortest_path_length(G, orig_node, dest_node, weight='length')

In [ ]:
# and we visualize it on the map
fig, ax = ox.plot_graph_route(G, shortest_path, route_color='red', route_linewidth=3, node_size=0)
print(f"Shortest path distance: {length:.2f} meters")

In [ ]:
# Get coordinates of each node in the path
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
route_coords

In [ ]:
# Center the map near the origin
m = folium.Map(location=route_coords[0], zoom_start=15)

# Add the route as a line
folium.PolyLine(route_coords, color="red", weight=5, opacity=0.8).add_to(m)

# Add origin and destination markers
folium.Marker(location=route_coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=route_coords[-1], popup="End", icon=folium.Icon(color='blue')).add_to(m)

m